In [2]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [3]:
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

In [4]:
article_text = "वैज्ञानिकों जॉर्जिया मिला तस्दीक 8,000 साल पुराने मिट्टी मर्तबान उसमें अंगूर शराब बनाए सबूत शोधकर्ताओं राय अंगूर शराब बनाने सबसे शुरुआती प्रमाण जॉर्जिया राजधानी तबलिसी दक्षिणी इलाकों जगहों मिले हैं.ये जगहें नवपाषाण युग संबंधित शराब अवशेष मिले मर्तबानों अंगूर गुच्छे नाचते शख़्स तस्वीर है.जॉर्जिया मिली चीज़ों बारे प्रोसीडिंग्स ऑफ़ नेशनल एकैडमी ऑफ़ साइसेंज रिसर्च रिपोर्ट छपी है.शराबी पतलून निकला अजगर ... शराब पीने लोग अंग्रेज़ी बोलते टोरंटो यूनिवर्सिटी सीनियर रिसर्चर रिसर्च सह-लेखक स्टीफ़न बाट्युक मानना जंगल उगने यूरेशियाई अंगूरों शराब बनाने सबसे पुराना उदाहरण स्टीफ़न राय जानते पश्चिम सभ्यताओं शराब ख़ास दवा तौर सामाजिक मेलजोल बहलाने शराब समाज अर्थव्यवस्था दवाओं खान-पान केंद्र इससे शराब बनाने सबसे पुराने प्रमाण मिले ईरान पाए ईरान मिले शराब मर्तबानों उम्र सात हज़ार साल बताई थी.साल अर्मेनिया गुफा हज़ार पुरानी शराब अवशेष मिले दुनिया सबसे पुरानी बिना अंगूर वाली शराब बारे सात हज़ार साल पुरानी चीन मिली शराब चावल शहद फलों बीबीसी हिन्दी एंड्रॉएड क्लिक हमें फ़ेसबुक ट्विटर फ़ॉलो"

In [5]:
model_name = "csebuetnlp/mT5_m2o_hindi_crossSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/Users/pranavdeepak/miniforge3/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [6]:
input_ids = tokenizer(
    [WHITESPACE_HANDLER(article_text)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    max_length=84,
    no_repeat_ngram_size=2,
    num_beams=4
)[0]

In [7]:
summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)
print(summary)

शोधकर्ताओं का कहना है कि उन्हें इसराइल में हैफ़ा के नज़दीक प्रागैतिहासिक काल की एक गुफा से दुनिया की सबसे पुरानी शराब की भट्टी मिली है. साथ ही 13,000 साल बाद पीने वाली चीज़ें भी इसकी चपेट में हैं.


In [8]:
import pickle 
# pickle th model 
with open('./models/TS_model.pkl', 'wb') as f:
    pickle.dump(model, f)

# pickle the tokenizer
with open('./models/TS_tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

In [9]:
# Write the above code as function which take in a article text and return summary
# Load the model and tokenizer from pickle file and use it to generate summary
# Path: TS.py
import re
import pickle
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

model3 = pickle.load(open("./models/TS_model.pkl", "rb"))
tokenizer3 = pickle.load(open("./models/TS_tokenizer.pkl", "rb"))

def text_summarizer(article_text):

    input_ids = tokenizer3(
        [WHITESPACE_HANDLER(article_text)],
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=512
    )["input_ids"]

    output_ids = model3.generate(
        input_ids=input_ids,
        max_length=84,
        no_repeat_ngram_size=2,
        num_beams=4
    )[0]

    summary = tokenizer3.decode(
        output_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )
    return summary

In [10]:
text_summarizer(article_text)

'शोधकर्ताओं का कहना है कि उन्हें इसराइल में हैफ़ा के नज़दीक प्रागैतिहासिक काल की एक गुफा से दुनिया की सबसे पुरानी शराब की भट्टी मिली है. साथ ही 13,000 साल बाद पीने वाली चीज़ें भी इसकी चपेट में हैं.'

In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW
import pandas as pd
# Load tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("csebuetnlp/mT5_m2o_hindi_crossSum")
model = T5ForConditionalGeneration.from_pretrained("csebuetnlp/mT5_m2o_hindi_crossSum")

# Prepare your dataset and DataLoader (not shown here)

data = pd.read_csv("./dataset/preprocessed_text.csv")
data = data.dropna()
data = data.reset_index(drop=True)
data = data.iloc[:1000]
data = data[data['text'].str.len() < 512]
data = data[data['text'].str.len() > 100]
data = data.reset_index(drop=True)
# Add the summarization column
data['summary'] = None

# Define a custom Dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, data):
        self.tokenizer = tokenizer
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        text = row['text']
        summary = row['summary']

        source_encoding = tokenizer(
            [WHITESPACE_HANDLER(text)],
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=512
        )

        target_encoding = tokenizer(
            [WHITESPACE_HANDLER(summary)],
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=150
        )

        labels = target_encoding["input_ids"]
        labels[labels[:, :] == 0] = -100

        return {
            "source_ids": source_encoding["input_ids"].flatten(),
            "source_mask": source_encoding["attention_mask"].flatten(),
            "target_ids": target_encoding["input_ids"].flatten(),
            "target_mask": target_encoding["attention_mask"].flatten(),
            "labels": labels.flatten()
        }
    
# Create a DataLoader for training and validation
train_dataset = CustomDataset(tokenizer, data)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)

# Define training parameters
learning_rate = 1e-5
epochs = 3

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

# Start training loop
for epoch in range(epochs):
    print(f"Epoch {epoch + 1} of {epochs}")
    model.train()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    epoch_loss = 0
    for batch, data in enumerate(train_dataloader):
        optimizer.zero_grad()
        source_ids = data['source_ids'].to(device, dtype=torch.long)
        source_mask = data['source_mask'].to(device, dtype=torch.long)
        target_ids = data['target_ids'].to(device, dtype=torch.long)
        target_mask = data['target_mask'].to(device, dtype=torch.long)
        labels = data['labels'].to(device, dtype=torch.long)

        outputs = model(
            input_ids=source_ids,
            attention_mask=source_mask,
            labels=labels,
            decoder_attention_mask=target_mask,
            use_cache=False
        )

        loss = outputs[0]
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

        if batch % 100 == 0:
            print(f"Iteration {batch} of {len(train_dataloader)} loss: {loss.item()}")

    print(f"Epoch {epoch + 1} average loss: {epoch_loss / len(train_dataloader)}")
    torch.save(model.state_dict(), f"t5_summary_{epoch + 1}.pth")

# Save trained model
model.save_pretrained("t5_summary")